In [1]:
import cudf
import os

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
df = cudf.read_parquet('../data/all_states_lat_long_education_age_sex.parquet/*')[['GISJOIN', 'p_id', 'sex' ,'age']]

In [4]:
df = df.query('age >= 16')

In [5]:
df['GISJOIN_og'] = df.GISJOIN
df['GISJOIN'] = (df.GISJOIN/1000).astype('int')

In [6]:
df

,GISJOIN,p_id,sex,age,GISJOIN_og
0,1000300103003,68800,0,16,1000300103003128
2,1000300103003,68802,0,17,1000300103003128
4,1000300103003,68804,0,16,1000300103003128
6,1000300103003,68806,0,16,1000300103003128
9,1000300103003,68809,0,16,1000300103003128
...,...,...,...,...,...
308745304,56004509513003,308744694,0,35,56004509513003072
308745305,56004509513003,308744695,0,35,56004509513003072
308745306,56004509513003,308744696,0,38,56004509513003072
308745307,56004509513003,308744697,0,35,56004509513003072


In [7]:
df.dtypes

GISJOIN       int64
p_id          int32
sex            int8
age            int8
GISJOIN_og    int64
dtype: object

## Adding income block wise - Male & Female

<b>JQVE001</b>:     Total<br>
<b>JQVE002</b>:     Male<br>
<b>JQVE003</b>:     Male: `$1 to $2,499 or loss`<br>
<b>JQVE004</b>:     Male: `$2,500 to $4,999`<br>
<b>JQVE005</b>:     Male: `$5,000 to $7,499`<br>
<b>JQVE006</b>:     Male: `$7,500 to $9,999`<br>
<b>JQVE007</b>:     Male: `$10,000 to $12,499`<br>
<b>JQVE008</b>:     Male: `$12,500 to $14,999`<br>
<b>JQVE009</b>:     Male: `$15,000 to $17,499`<br>
<b>JQVE010</b>:     Male: `$17,500 to $19,999`<br>
<b>JQVE011</b>:     Male: `$20,000 to $22,499`<br>
<b>JQVE012</b>:     Male: `$22,500 to $24,999`<br>
<b>JQVE013</b>:     Male: `$25,000 to $29,999`<br>
<b>JQVE014</b>:     Male: `$30,000 to $34,999`<br>
<b>JQVE015</b>:     Male: `$35,000 to $39,999`<br>
<b>JQVE016</b>:     Male: `$40,000 to $44,999`<br>
<b>JQVE017</b>:     Male: `$45,000 to $49,999`<br>
<b>JQVE018</b>:     Male: `$50,000 to $54,999`<br>
<b>JQVE019</b>:     Male: `$55,000 to $64,999`<br>
<b>JQVE020</b>:     Male: `$65,000 to $74,999`<br>
<b>JQVE021</b>:     Male: `$75,000 to $99,999`<br>
<b>JQVE022</b>:     Male: `$100,000 or more `<br>


<b>JQVE023</b>:     Female<br>
<b>JQVE024</b>:     Female: `$1 to $2,499 or loss`<br>
<b>JQVE025</b>:     Female: `$2,500 to $4,999`<br>
<b>JQVE026</b>:     Female: `$5,000 to $7,499`<br>
<b>JQVE027</b>:     Female: `$7,500 to $9,999`<br>
<b>JQVE028</b>:     Female: `$10,000 to $12,499`<br>
<b>JQVE029</b>:     Female: `$12,500 to $14,999`<br>
<b>JQVE030</b>:     Female: `$15,000 to $17,499`<br>
<b>JQVE031</b>:     Female: `$17,500 to $19,999`<br>
<b>JQVE032</b>:     Female: `$20,000 to $22,499`<br>
<b>JQVE033</b>:     Female: `$22,500 to $24,999`<br>
<b>JQVE034</b>:     Female: `$25,000 to $29,999`<br>
<b>JQVE035</b>:     Female: `$30,000 to $34,999`<br>
<b>JQVE036</b>:     Female: `$35,000 to $39,999`<br>
<b>JQVE037</b>:     Female: `$40,000 to $44,999`<br>
<b>JQVE038</b>:     Female: `$45,000 to $49,999`<br>
<b>JQVE039</b>:     Female: `$50,000 to $54,999`<br>
<b>JQVE040</b>:     Female: `$55,000 to $64,999`<br>
<b>JQVE041</b>:     Female: `$65,000 to $74,999`<br>
<b>JQVE042</b>:     Female: `$75,000 to $99,999`<br>
<b>JQVE043</b>:     Female: `$100,000 or more`<br>

In [8]:
df_cat = cudf.read_csv('/home/ajay/data/census/all_states_block_group/nhgis0016_csv/nhgis0016_ds176_20105_2010_blck_grp.csv')

df_cat.STATEA = df_cat.STATEA.astype('int')
df_cat = df_cat.query('STATEA != 72')

df_cat.GISJOIN = df_cat.GISJOIN.str.replace('G', '').astype('int')

sex_by_income_cols = ['JQVE002','JQVE003','JQVE004','JQVE005','JQVE006','JQVE007','JQVE008','JQVE009','JQVE010','JQVE011','JQVE012','JQVE013','JQVE014','JQVE015','JQVE016','JQVE017','JQVE018','JQVE019','JQVE020','JQVE021','JQVE022','JQVE023','JQVE024','JQVE025','JQVE026','JQVE027','JQVE028','JQVE029','JQVE030','JQVE031','JQVE032','JQVE033','JQVE034','JQVE035', 'JQVE036','JQVE037','JQVE038','JQVE039','JQVE040','JQVE041','JQVE042','JQVE043']

sex_by_income_cols_male = ['JQVE002','JQVE003','JQVE004','JQVE005','JQVE006','JQVE007','JQVE008','JQVE009','JQVE010','JQVE011','JQVE012','JQVE013','JQVE014','JQVE015','JQVE016','JQVE017','JQVE018', 'JQVE019','JQVE020','JQVE021','JQVE022']
sex_by_income_cols_female = ['JQVE023','JQVE024','JQVE025','JQVE026','JQVE027','JQVE028','JQVE029','JQVE030','JQVE031','JQVE032','JQVE033','JQVE034','JQVE035', 'JQVE036','JQVE037','JQVE038','JQVE039','JQVE040','JQVE041','JQVE042','JQVE043']

df_cat = df_cat[['GISJOIN']+sex_by_income_cols]

for i in df_cat.columns:
    if i != 'GISJOIN':
        df_cat[i] = df_cat[i].astype('int16')
        
df_cat = df_cat.fillna(0)

In [9]:
df_cat

,GISJOIN,JQVE002,JQVE003,JQVE004,JQVE005,JQVE006,JQVE007,JQVE008,JQVE009,JQVE010,...,JQVE034,JQVE035,JQVE036,JQVE037,JQVE038,JQVE039,JQVE040,JQVE041,JQVE042,JQVE043
0,1000100201001,126,0,0,0,0,0,0,0,0,...,42,21,10,15,0,13,0,13,0,26
1,1000100201002,374,35,0,5,0,18,0,0,36,...,45,22,12,31,17,26,0,29,0,0
2,1000100202001,213,3,18,0,13,19,25,12,21,...,53,13,18,13,0,0,0,0,19,0
3,1000100202002,236,0,0,0,12,10,20,4,0,...,4,38,11,18,20,5,0,0,0,8
4,1000100203001,824,51,40,49,40,21,33,0,8,...,66,56,30,0,0,14,5,21,29,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217735,56004509511001,500,23,26,19,1,7,27,0,0,...,28,26,29,3,7,20,3,15,0,8
217736,56004509511002,544,39,17,9,14,3,25,8,2,...,21,0,15,6,0,30,28,0,31,26
217737,56004509513001,486,13,0,33,0,0,0,11,0,...,82,35,15,24,0,0,17,0,0,0
217738,56004509513002,305,4,0,11,0,6,20,13,0,...,25,18,0,9,10,9,2,0,0,0


In [10]:
all(df_cat[sex_by_income_cols_male[1:]].to_pandas().sum(axis=1).values == df_cat.JQVE002.to_pandas().values)

True

In [11]:
all(df_cat[sex_by_income_cols_female[1:]].to_pandas().sum(axis=1).values == df_cat.JQVE023.to_array())

True

In [12]:
for i in sex_by_income_cols_male:
    if i != 'JQVE002':
        df_cat[i] = df_cat[i]/df_cat['JQVE002'].astype('float32')#*100).floor().astype('int16')

for i in sex_by_income_cols_female:
    if i != 'JQVE023':
        df_cat[i] = df_cat[i]/df_cat['JQVE023'].astype('float32')#*100).floor().astype('int16')
        
df_cat.drop_column('JQVE002')
df_cat.drop_column('JQVE023')

In [13]:
df_cat

,GISJOIN,JQVE003,JQVE004,JQVE005,JQVE006,JQVE007,JQVE008,JQVE009,JQVE010,JQVE011,...,JQVE034,JQVE035,JQVE036,JQVE037,JQVE038,JQVE039,JQVE040,JQVE041,JQVE042,JQVE043
0,1000100201001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.119048,...,0.238636,0.119318,0.056818,0.085227,0.000000,0.073864,0.000000,0.073864,0.000000,0.147727
1,1000100201002,0.093583,0.000000,0.013369,0.000000,0.048128,0.000000,0.000000,0.096257,0.000000,...,0.127479,0.062323,0.033994,0.087819,0.048159,0.073654,0.000000,0.082153,0.000000,0.000000
2,1000100202001,0.014085,0.084507,0.000000,0.061033,0.089202,0.117371,0.056338,0.098592,0.000000,...,0.177258,0.043478,0.060201,0.043478,0.000000,0.000000,0.000000,0.000000,0.063545,0.000000
3,1000100202002,0.000000,0.000000,0.000000,0.050847,0.042373,0.084746,0.016949,0.000000,0.000000,...,0.017241,0.163793,0.047414,0.077586,0.086207,0.021552,0.000000,0.000000,0.000000,0.034483
4,1000100203001,0.061893,0.048544,0.059466,0.048544,0.025485,0.040049,0.000000,0.009709,0.065534,...,0.108197,0.091803,0.049180,0.000000,0.000000,0.022951,0.008197,0.034426,0.047541,0.021311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217735,56004509511001,0.046000,0.052000,0.038000,0.002000,0.014000,0.054000,0.000000,0.000000,0.034000,...,0.083832,0.077844,0.086826,0.008982,0.020958,0.059880,0.008982,0.044910,0.000000,0.023952
217736,56004509511002,0.071691,0.031250,0.016544,0.025735,0.005515,0.045956,0.014706,0.003676,0.027574,...,0.059829,0.000000,0.042735,0.017094,0.000000,0.085470,0.079772,0.000000,0.088319,0.074074
217737,56004509513001,0.026749,0.000000,0.067901,0.000000,0.000000,0.000000,0.022634,0.000000,0.018519,...,0.200980,0.085784,0.036765,0.058824,0.000000,0.000000,0.041667,0.000000,0.000000,0.000000
217738,56004509513002,0.013115,0.000000,0.036066,0.000000,0.019672,0.065574,0.042623,0.000000,0.042623,...,0.105042,0.075630,0.000000,0.037815,0.042017,0.037815,0.008403,0.000000,0.000000,0.000000


### merge df_cat's index with df

In [14]:
df = df.merge(df_cat[['GISJOIN']], on='GISJOIN')

In [15]:
df

,GISJOIN,p_id,sex,age,GISJOIN_og
0,1000100208012,32068,1,56,1000100208012069
1,1000100208012,32069,1,58,1000100208012069
2,1000100208012,32070,1,57,1000100208012069
3,1000100208012,32071,1,57,1000100208012069
4,1000100208012,32072,1,59,1000100208012069
...,...,...,...,...,...
243227903,56004509511001,308738193,0,25,56004509511001056
243227904,56004509511001,308738194,0,26,56004509511001056
243227905,56004509511001,308738195,0,29,56004509511001056
243227906,56004509511001,308738196,0,25,56004509511001056


#### Make sure total ratio is a complete 1 (or 0 incase there are no values)

In [16]:
df1 = df.groupby(by=['GISJOIN', 'sex'], as_index=False).count()

In [17]:
df1 = df1.one_hot_encoding('sex', 'gender', [0,1])

In [18]:
df1.drop_column('sex')

In [19]:
df1.gender_0 = (df1.GISJOIN_og*df1.gender_0).astype('int')
df1.gender_1 = (df1.GISJOIN_og*df1.gender_1).astype('int')

In [20]:
df1.drop_column('GISJOIN_og')

In [21]:
df1 = df1.groupby('GISJOIN', as_index=False).sum()
df1

,GISJOIN,p_id,age,gender_0,gender_1
0,1000100201001,575,575,281,294
1,1000100201002,929,929,423,506
2,1000100202001,734,734,292,442
3,1000100202002,921,921,483,438
4,1000100203001,1840,1840,739,1101
...,...,...,...,...,...
216859,56004509511001,1279,1279,698,581
216860,56004509511002,1548,1548,906,642
216861,56004509513001,943,943,402,541
216862,56004509513002,1118,1118,518,600


#### Merge population in df with ratios in df_cat

In [22]:
df1 = df1.merge(df_cat, on='GISJOIN')

In [23]:
df1

,GISJOIN,p_id,age,gender_0,gender_1,JQVE003,JQVE004,JQVE005,JQVE006,JQVE007,...,JQVE034,JQVE035,JQVE036,JQVE037,JQVE038,JQVE039,JQVE040,JQVE041,JQVE042,JQVE043
0,6003100008002,1888,1888,976,912,0.058954,0.007786,0.086763,0.056730,0.024472,...,0.000000,0.038732,0.028169,0.017606,0.054577,0.014085,0.000000,0.028169,0.014085,0.054577
1,6003100008003,532,532,247,285,0.041494,0.058091,0.000000,0.029046,0.186722,...,0.113402,0.041237,0.103093,0.015464,0.025773,0.015464,0.000000,0.000000,0.041237,0.020619
2,6003100008004,388,388,183,205,0.301370,0.036530,0.041096,0.105023,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.086957,0.086957,0.119565,0.000000
3,6003100009001,699,699,416,283,0.000000,0.000000,0.000000,0.128342,0.320856,...,0.048193,0.283133,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,6003100009002,564,564,257,307,0.000000,0.000000,0.073232,0.000000,0.000000,...,0.000000,0.066667,0.000000,0.000000,0.000000,0.070588,0.074510,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,1111,1111,558,553,0.023136,0.084833,0.051414,0.020566,0.000000,...,0.073418,0.045570,0.088608,0.060759,0.045570,0.040506,0.037975,0.032911,0.025316,0.017722
216860,55000709604001,1228,1228,612,616,0.095528,0.026423,0.050813,0.016260,0.022358,...,0.108235,0.141176,0.047059,0.044706,0.037647,0.044706,0.035294,0.011765,0.007059,0.000000
216861,55000709604002,582,582,331,251,0.000000,0.084848,0.024242,0.030303,0.024242,...,0.054878,0.091463,0.024390,0.060976,0.054878,0.018293,0.048780,0.012195,0.036585,0.000000
216862,55000709604003,722,722,363,359,0.059259,0.033333,0.040741,0.003704,0.022222,...,0.080717,0.094170,0.022422,0.026906,0.071749,0.008969,0.062780,0.022422,0.000000,0.000000


In [24]:
#males
df1.to_pandas().query('JQVE003.isnull()')

,GISJOIN,p_id,age,gender_0,gender_1,JQVE003,JQVE004,JQVE005,JQVE006,JQVE007,...,JQVE034,JQVE035,JQVE036,JQVE037,JQVE038,JQVE039,JQVE040,JQVE041,JQVE042,JQVE043
392,6003705041021,22,22,0,22,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1683,4001300405293,223,223,60,163,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.000,0.0
1688,4001300405312,829,829,389,440,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.000,0.0
2075,6005309800001,7,7,0,7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2710,12008300018002,426,426,113,313,NaN,NaN,NaN,NaN,NaN,...,0.333333,0.251462,0.0,0.0000,0.0,0.0,0.0,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213458,51013509801001,34,34,0,34,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213991,54006100101012,176,176,0,176,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.0,0.3125,0.0,0.0,0.0,0.0,0.375,0.0
215037,51005909801001,10,10,0,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215038,51005909803001,4,4,0,4,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#females
df1.to_pandas().query('JQVE024.isnull()')

,GISJOIN,p_id,age,gender_0,gender_1,JQVE003,JQVE004,JQVE005,JQVE006,JQVE007,...,JQVE034,JQVE035,JQVE036,JQVE037,JQVE038,JQVE039,JQVE040,JQVE041,JQVE042,JQVE043
392,6003705041021,22,22,0,22,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2047,6005300141022,156,156,156,0,0.000000,0.250000,0.000000,0.326923,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2075,6005309800001,7,7,0,7,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2588,6005900761022,2632,2632,2499,133,0.170429,0.113995,0.104966,0.029345,0.107223,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2664,6003500404002,8126,8126,8120,6,0.211009,0.138925,0.022280,0.073394,0.085190,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209881,51018700202002,149,149,149,0,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
211027,55000900211001,1090,1090,1090,0,0.337500,0.100000,0.125000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
213458,51013509801001,34,34,0,34,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215037,51005909801001,10,10,0,10,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
df2 = df1.copy()

In [28]:
null_replace_equal_distribute_factor = (100/len(sex_by_income_cols_male[1:]))/100
null_replace_equal_distribute_factor

0.05

In [29]:
#males
for i in sex_by_income_cols_male[1:]:
    df1[i] = ((df1[i] * df1.gender_0)).floor().astype('int16')
    df2[i] = ((df2[i] * df2.gender_0) - df1[i]).astype('float32')
    
t = df2.to_pandas().query('JQVE003.isnull()').index #everything is null for a row in this case

for i in sex_by_income_cols_male[1:]:
    df1[i].loc[[t]] = 0
    df2[i].loc[[t]] = null_replace_equal_distribute_factor

df1['diff_male'] = df1.gender_0.to_pandas() - df1[sex_by_income_cols_male[1:]].to_pandas().sum(axis=1)

In [30]:
#females
# df2 = df1.copy()
for i in sex_by_income_cols_female[1:]:
    df1[i] = ((df1[i] * df1.gender_1)).floor().astype('int16')
    df2[i] = ((df2[i] * df2.gender_1) - df1[i]).astype('float32')
    
t = df2.to_pandas().query('JQVE024.isnull()').index #everything is null for a row in this case

for i in sex_by_income_cols_female[1:]:
    df1[i].loc[[t]] = 0
    df2[i].loc[[t]] = null_replace_equal_distribute_factor

df1['diff_female'] = df1.gender_1.to_pandas() - df1[sex_by_income_cols_female[1:]].to_pandas().sum(axis=1)

In [31]:
df1

,GISJOIN,p_id,age,gender_0,gender_1,JQVE003,JQVE004,JQVE005,JQVE006,JQVE007,...,JQVE036,JQVE037,JQVE038,JQVE039,JQVE040,JQVE041,JQVE042,JQVE043,diff_male,diff_female
0,6003100008002,1888,1888,976,912,57,7,84,55,23,...,25,16,49,12,0,25,12,49,11,9
1,6003100008003,532,532,247,285,10,14,0,7,46,...,29,4,7,4,0,0,11,5,5,10
2,6003100008004,388,388,183,205,55,6,7,19,0,...,0,0,0,0,17,17,24,0,4,3
3,6003100009001,699,699,416,283,0,0,0,53,133,...,0,0,0,0,0,0,0,0,3,2
4,6003100009002,564,564,257,307,0,0,18,0,0,...,0,0,0,21,22,0,0,0,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,1111,1111,558,553,12,47,28,11,0,...,48,33,25,22,20,18,14,9,11,10
216860,55000709604001,1228,1228,612,616,58,16,31,9,13,...,28,27,23,27,21,7,4,0,9,10
216861,55000709604002,582,582,331,251,0,28,8,10,8,...,6,15,13,4,12,3,9,0,1,9
216862,55000709604003,722,722,363,359,21,12,14,1,8,...,8,9,25,3,22,8,0,0,6,8


In [32]:
df1.diff_male.max()

3235

In [33]:
df2[sex_by_income_cols_male[1:]].to_pandas().sum(axis=1).unique()

array([11.00000542,  5.00000513,  4.00000237, ..., 11.00000869,
        1.00000247, 10.00002168])

In [34]:
print(df1.diff_male.sum() == (df1.gender_0.to_array().sum() - df1[sex_by_income_cols_male[1:]].to_pandas().sum(axis=1).values.sum()))
print(df1.diff_female.sum() == (df1.gender_1.to_array().sum() - df1[sex_by_income_cols_female[1:]].to_pandas().sum(axis=1).values.sum()))

True
True


In [35]:
print(df1.gender_0.to_array().sum() == df1[sex_by_income_cols_male[1:]].to_pandas().sum(axis=1).values.sum())
print(df1.gender_1.to_array().sum() == df1[sex_by_income_cols_female[1:]].to_pandas().sum(axis=1).values.sum())

False
False


#### filling the difference

Since they don't add up to the total population, lets apply the largest remainder method, mentioned here -> https://revs.runtime-revolution.com/getting-100-with-rounded-percentages-273ffa70252b

In [36]:
import cupy

cp_array_male = cupy.array(df2[sex_by_income_cols_male[1:]].to_gpu_matrix())
cp_array_male = cupy.nan_to_num(cp_array_male/ cp_array_male.sum(axis=1, keepdims=True))
cp_array_male_sorted = cupy.asnumpy(cp_array_male.argsort())
cp_array_male = cupy.asnumpy(cp_array_male)

cp_array_female = cupy.array(df2[sex_by_income_cols_female[1:]].to_gpu_matrix())
cp_array_female = cupy.nan_to_num(cp_array_female/ cp_array_female.sum(axis=1, keepdims=True))
cp_array_female_sorted = cupy.asnumpy(cp_array_female.argsort())
cp_array_female = cupy.asnumpy(cp_array_female)

Handle remaining remainder values

In [37]:
from multiprocessing import  Pool
from functools import partial
import numpy as np
import pandas as pd

def parallelize(data, func, num_of_processes=8):
    data_split = np.array_split(data, num_of_processes)
    pool = Pool(num_of_processes)
    data = pd.concat(pool.map(func, data_split))
    pool.close()
    pool.join()
    return data

def run_on_subset(func, data_subset):
    return data_subset.apply(func, axis=1)

def parallelize_on_rows(data, func, num_of_processes=8):
    return parallelize(data, partial(run_on_subset, func), num_of_processes)

In [38]:
df1_pandas = df1.to_pandas()

count = 0

def applyrows(row):
    global count, cp_array_male_sorted, cp_array_male, cp_array_female_sorted, cp_array_female
    #males
    sorted_series_male = cp_array_male_sorted[row.name][::-1]
    i = row['diff_male']
    while i > 0:
        for sorted_index in sorted_series_male:
            sorted_val = cp_array_male[row.name][sorted_index]
            if sorted_val == 0:
                break
            if int(sorted_val * i) <= 1:
                t_value = i
            else:
                t_value = int(sorted_val * i)
            t_index = 'JQVE0'+str(sorted_index+3).zfill(2)
            row[t_index] += t_value
            i -= t_value
    #females
    sorted_series_female = cp_array_female_sorted[row.name][::-1]
    i = row['diff_female']
    while i > 0:
        for sorted_index in sorted_series_female:
            sorted_val = cp_array_female[row.name][sorted_index]
            if sorted_val == 0:
                break
            if int(sorted_val * i) <= 1:
                t_value = i
            else:
                t_value = int(sorted_val * i)
            t_index = 'JQVE0'+str(sorted_index+24).zfill(2)
            row[t_index] += t_value
            i -= t_value
    
    count += 1
    if count%10000 == 0: 
        print('                                  ', end='\r')
        print('count', count,row.name, end='\r')
    return row

# df2_pandas = df1_pandas.apply(applyrows, axis=1)
df2_pandas = parallelize_on_rows(df1_pandas, applyrows) 

In [39]:
df2_pandas

,GISJOIN,p_id,age,gender_0,gender_1,JQVE003,JQVE004,JQVE005,JQVE006,JQVE007,...,JQVE036,JQVE037,JQVE038,JQVE039,JQVE040,JQVE041,JQVE042,JQVE043,diff_male,diff_female
0,6003100008002,1888,1888,976,912,57,7,84,55,23,...,25,16,49,12,0,25,12,49,11,9
1,6003100008003,532,532,247,285,10,14,0,7,46,...,29,4,7,4,0,0,11,15,5,10
2,6003100008004,388,388,183,205,55,6,7,19,0,...,0,0,0,0,17,20,24,0,4,3
3,6003100009001,699,699,416,283,0,0,0,53,133,...,0,0,0,0,0,0,0,0,3,2
4,6003100009002,564,564,257,307,0,0,18,0,0,...,0,0,0,21,26,0,0,0,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,1111,1111,558,553,12,47,28,11,0,...,58,33,25,22,20,18,14,9,11,10
216860,55000709604001,1228,1228,612,616,58,16,31,9,13,...,38,27,23,27,21,7,4,0,9,10
216861,55000709604002,582,582,331,251,0,28,8,10,8,...,6,15,13,4,12,3,9,0,1,9
216862,55000709604003,722,722,363,359,21,12,14,1,8,...,8,9,25,3,22,8,0,0,6,8


In [40]:
print(df2_pandas.gender_0.values.sum() == df2_pandas[sex_by_income_cols_male[1:]].sum(axis=1).values.sum())
print(df2_pandas.gender_1.values.sum() == df2_pandas[sex_by_income_cols_female[1:]].sum(axis=1).values.sum())

True
True


In [41]:
df_final = cudf.from_pandas(df2_pandas)

In [42]:
df_final

,GISJOIN,p_id,age,gender_0,gender_1,JQVE003,JQVE004,JQVE005,JQVE006,JQVE007,...,JQVE036,JQVE037,JQVE038,JQVE039,JQVE040,JQVE041,JQVE042,JQVE043,diff_male,diff_female
0,6003100008002,1888,1888,976,912,57,7,84,55,23,...,25,16,49,12,0,25,12,49,11,9
1,6003100008003,532,532,247,285,10,14,0,7,46,...,29,4,7,4,0,0,11,15,5,10
2,6003100008004,388,388,183,205,55,6,7,19,0,...,0,0,0,0,17,20,24,0,4,3
3,6003100009001,699,699,416,283,0,0,0,53,133,...,0,0,0,0,0,0,0,0,3,2
4,6003100009002,564,564,257,307,0,0,18,0,0,...,0,0,0,21,26,0,0,0,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216859,55000709603003,1111,1111,558,553,12,47,28,11,0,...,58,33,25,22,20,18,14,9,11,10
216860,55000709604001,1228,1228,612,616,58,16,31,9,13,...,38,27,23,27,21,7,4,0,9,10
216861,55000709604002,582,582,331,251,0,28,8,10,8,...,6,15,13,4,12,3,9,0,1,9
216862,55000709604003,722,722,363,359,21,12,14,1,8,...,8,9,25,3,22,8,0,0,6,8


In [43]:
print(df_final.gender_0.to_array().sum() == df_final[sex_by_income_cols_male[1:]].to_pandas().sum(axis=1).values.sum())
print(df_final.gender_1.to_array().sum() == df_final[sex_by_income_cols_female[1:]].to_pandas().sum(axis=1).values.sum())

True
True


In [44]:
df_final.drop_column('diff_male')
df_final.drop_column('diff_female')
# df_final.drop_column('GISJOIN')
# df_final.rename({'GISJOIN_og':'GISJOIN'}, inplace=True)

In [45]:
df_final.to_parquet('income_step_1_final.parquet')

/home/ajay/anaconda3/envs/rapids_dev_0.12/lib/python3.6/site-packages/cudf/io/parquet.py:70: UserWarning: Using CPU via PyArrow to write Parquet dataset, this will be GPU accelerated in the future
  "Using CPU via PyArrow to write Parquet dataset, this will "


In [46]:
df.to_parquet('GISJOIN_to_GISJOIN_og_mapping.parquet')